In [355]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sm
# Any results you write to the current directory are saved as output.

In [356]:
#Считаем данные
sample= pd.read_csv("sample_submission.csv")
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [357]:
cols = train.columns
train_columns = cols[1:-2]
target_columns = cols[-1]
###
train_columns = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [358]:
#выкидываем строки без информации о возрасте из обучающей выборки
train_first = train[train_columns].dropna()
#train_first = train[train_columns].copy()

target_first = train[target_columns]
target_first = target_first[train_first.index]

In [359]:
#поля пола и пути назначения переведем в целые числа
sex_map = {'male' : 1, 'female': 2}
emb_map = {'C':1, 'Q':2,'S':3}
train_first['Sex'] = train['Sex'].map(sex_map)
train_first['Embarked'] = train['Embarked'].map(emb_map)


# for col in train_first.columns:
#     train_first[col].fillna(train_first[col].mean(), inplace=True)

train_first['Sex'].fillna(train_first['Sex'].mean(), inplace=True)
train_first['Embarked'].fillna(train_first['Embarked'].mean(), inplace=True)
train_first['Fare'].fillna(train_first['Fare'].mean(), inplace=True)

In [360]:
#нормализуем данные из тренировочной выборки

train_first_scaled = (train_first - train_first.min(axis=0))/(train_first.max(axis=0) - train_first.min(axis=0))
#train_first_scaled = (train_first - train_first.mean(axis=0))/train_first.std(axis=0)


In [361]:
# разделим выборку
#X_train, X_test, y_train, y_test = train_test_split(train_first_scaled, target_first, test_size=0.2, random_state=42)

In [362]:
#используем логистическую регрессию

logreg_model = LogisticRegression()
logreg_model.fit(train_first_scaled, target_first)

c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from tqdm import tqdm

MAX_EMPTY_ATTEMPTS = 10000

C_values = np.arange(1, 10000, 10)

best_score = 0
best_param = 0.1
best_model = None

empty_attempts = 0
for C in tqdm(C_values):
    model = LogisticRegression(C=C)
    score = np.mean(cross_val_score(model, train_first_scaled, target_first, cv=5, scoring="accuracy"))
    
    if score > best_score:
        best_score = score
        best_model = model
        best_param = C
        
        empty_attempts = 0 
    else:
        empty_attempts += 1
        
    if empty_attempts == MAX_EMPTY_ATTEMPTS:
        print("Early finish")
        break
        
    
    
print(f"Hyperparameters selection finished, best score - {best_score:0.4f} at C={best_param}")
best_model.fit(train_first_scaled, target_first)

In [354]:
print(vars(logreg_model))
print(vars(best_model))

{'penalty': 'l2', 'dual': False, 'tol': 0.0001, 'C': 1.0, 'fit_intercept': True, 'intercept_scaling': 1, 'class_weight': None, 'random_state': None, 'solver': 'warn', 'max_iter': 100, 'multi_class': 'warn', 'verbose': 0, 'warm_start': False, 'n_jobs': None, 'l1_ratio': None, 'classes_': array([0, 1], dtype=int64), 'coef_': array([[-1.71450955,  2.40062718, -1.6812256 , -0.77623021,  0.45741052,
         0.45455956, -0.41066761]]), 'intercept_': array([0.62357234]), 'n_iter_': array([5], dtype=int32)}
{'penalty': 'l2', 'dual': False, 'tol': 0.0001, 'C': 1, 'fit_intercept': True, 'intercept_scaling': 1, 'class_weight': None, 'random_state': None, 'solver': 'warn', 'max_iter': 100, 'multi_class': 'warn', 'verbose': 0, 'warm_start': False, 'n_jobs': None, 'l1_ratio': None, 'classes_': array([0, 1], dtype=int64), 'coef_': array([[-1.71450955,  2.40062718, -1.6812256 , -0.77623021,  0.45741052,
         0.45455956, -0.41066761]]), 'intercept_': array([0.62357234]), 'n_iter_': array([5], dtyp

In [363]:
#формируем предсказания для тестовой выборки и оценеваем ошибки
preds = logreg_model.predict(X_test)

print(sm.classification_report(y_test, preds, digits=5))


              precision    recall  f1-score   support

           0    0.83784   0.78481   0.81046        79
           1    0.73016   0.79310   0.76033        58

    accuracy                        0.78832       137
   macro avg    0.78400   0.78896   0.78539       137
weighted avg    0.79225   0.78832   0.78924       137



In [365]:
cv_acc_scoring = cross_val_score(logreg_model, train_first_scaled, target_first, cv=5, scoring="accuracy")

print(f"5 folds accuracy: avg - {np.mean(cv_acc_scoring):.2f}, std - {np.std(cv_acc_scoring):.2f} - {cv_acc_scoring}")


cv_prec_scoring = cross_val_score(logreg_model, train_first_scaled, target_first, cv=5, scoring="precision")

print(f"5 folds precision: avg - {np.mean(cv_prec_scoring):.2f}, std - {np.std(cv_prec_scoring):.2f} - {cv_prec_scoring:}")

cv_f_scoring = cross_val_score(logreg_model, train_first_scaled, target_first, cv=5, scoring="f1")

print(f"5 folds f1: avg - {np.mean(cv_f_scoring):.2f}, std - {np.std(cv_f_scoring):.2f} - {cv_f_scoring:}")

c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\pyt

5 folds accuracy: avg - 0.78, std - 0.03 - [0.8115942  0.73188406 0.77372263 0.81617647 0.75735294]


c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\pyt

5 folds precision: avg - 0.74, std - 0.05 - [0.76785714 0.67272727 0.72222222 0.8125     0.73913043]


c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\артём\appdata\local\programs\python\pyt

5 folds f1: avg - 0.72, std - 0.04 - [0.76785714 0.66666667 0.71559633 0.75728155 0.67326733]


#### Стандартная нормализация, выкидывание нан столбцов из обучающей выборки
    5 folds accuracy: avg - 0.78, std - 0.03 - [0.8115942  0.73188406 0.77372263 0.80882353 0.76470588]
    5 folds precision: avg - 0.74, std - 0.04 - [0.76785714 0.67272727 0.73076923 0.79591837 0.73469388]
    5 folds f1: avg - 0.72, std - 0.04 - [0.76785714 0.66666667 0.71028037 0.75       0.69230769]]
    
#### Стандартная нормализация, кроме возраста, для которого минимаксная, выкидывание нан столбцов из обучающей выборки
    5 folds accuracy: avg - 0.77, std - 0.03 - [0.80434783 0.72463768 0.77372263 0.80147059 0.76470588]
    5 folds precision: avg - 0.74, std - 0.04 - [0.75438596 0.66666667 0.73076923 0.78       0.74468085]
    5 folds f1: avg - 0.71, std - 0.04 - [0.76106195 0.65454545 0.71028037 0.74285714 0.68627451]
#### Минисаксная нормализация, выкидывание нан столбцов из обучающей выборки   
    5 folds accuracy: avg - 0.78, std - 0.03 - [0.8115942  0.73188406 0.77372263 0.81617647 0.75735294]
    5 folds precision: avg - 0.74, std - 0.05 - [0.76785714 0.67272727 0.72222222 0.8125     0.73913043]
    5 folds f1: avg - 0.72, std - 0.04 - [0.76785714 0.66666667 0.71559633 0.75728155 0.67326733]
    
#### Стандартная нормализация, замена NaN столбцов средним значением
    5 folds accuracy: avg - 0.78, std - 0.03 - [0.83529412 0.76470588 0.77647059 0.77647059 0.75882353]
    5 folds precision: avg - 0.73, std - 0.03 - [0.79365079 0.69230769 0.71428571 0.72131148 0.71428571]
    5 folds f1: avg - 0.71, std - 0.04 - [0.78125    0.69230769 0.703125   0.6984127  0.66115702]
    
#### Минимаксная нормализация, замена NaN столбцов средним значением  

    5 folds accuracy: avg - 0.78, std - 0.03 - [0.83529412 0.76470588 0.77647059 0.77647059 0.75882353]
    5 folds precision: avg - 0.73, std - 0.03 - [0.79365079 0.69230769 0.71428571 0.72131148 0.71428571]
    5 folds f1: avg - 0.71, std - 0.04 - [0.78125    0.69230769 0.703125   0.6984127  0.66115702]

#### Стандартная нормализация, замена NaN столбцов средним значением, 1 2 3, 1 2
    5 folds accuracy: avg - 0.78, std - 0.03 - [0.8115942  0.73188406 0.77372263 0.81617647 0.75735294]
    5 folds precision: avg - 0.74, std - 0.05 - [0.76785714 0.67272727 0.72222222 0.8125     0.73913043]
    5 folds f1: avg - 0.72, std - 0.04 - [0.76785714 0.66666667 0.71559633 0.75728155 0.67326733]


In [150]:
# # проведем те же манипуляции с тестовыми данными

# test_first = test[train_columns].copy()

# test_first['Sex'] = test_first['Sex'].map(sex_map)
# test_first['Embarked'] = test_first['Embarked'].map(emb_map)

# test_first['Age'].fillna(test_first['Age'].mean(), inplace=True)
# test_first['Embarked'].fillna(test_first['Embarked'].mean(), inplace=True)

# test_first_scaled = (test_first - test_first.mean(axis=0))/test_first.std(axis=0)
# #для возраста используем минимаксную нормализацию
# test_first_scaled['Age'] = (test_first['Age'] - test_first['Age'].min())/(test_first['Age'].max() - test_first['Age'].min())

In [140]:
#формируем предсказания для тестовой выборки, используя полученную выше модель
preds = logreg_model.predict(X_test)

#записываем результат

sample['Survived'] = preds
sample.to_csv('first_solution.csv', index=False)

ValueError: Length of values does not match length of index